In [ ]:
!pip install googletrans==3.1.0a0

In [2]:
from googletrans import Translator
import pandas as pd

In [ ]:
description=pd.read_csv('raw_data/Clustering_ProductDescription.csv', engine='python', error_bad_lines=False, encoding='utf8')

description = description.fillna("null")

In [4]:
description

,ASIN,MARKETPLACE_NAME,SELLER_NAME,ITEM_DESCRIPTION
0,B0847TJHB1,United Kingdom,Espring,<b>&#10031&#10031 WINTEX Fabric Template Ruler...
1,B06XVJL3NP,United Kingdom,H&S,null
2,B01M8GI87J,Germany,Espring,<p><b>&#10031&#10031 PRETEX 5ER-SET MIKROFASER...
3,B08X1SQWL4,Italy,Tritart,null
4,B07D26LYLH,Poland,H&S,null
...,...,...,...,...
8637,B07NTY3D93,Germany,Tritart,<b>TRITART abwischbare Tafel-Aufkleber - selbs...
8638,B0875PQLS9,Poland,Espring,null
8639,B018ASUHVU,Germany,Espring,null
8640,B01J3WOZ4M,Poland,Espring,<p><strong>Wintex Notatnik A4 96 Kwadratowych ...


---
Detect description language and translate to english if necessary

In [5]:
translator = Translator()

lang_detect = lambda x : translator.detect(x).lang
to_english = lambda x: translator.translate(x, dest='en').text


In [7]:
#takes a while to execute. Haven't managed to optimize this line.

description['detected_lang'] = description['ITEM_DESCRIPTION'].apply(lang_detect)

In [ ]:
non_english_descriptions = description[description['detected_lang'] != "en"]
english_descriptions = description[description['detected_lang'] == "en"]
english_descriptions['detected_lang'] = "en"

In [ ]:
#takes a while to execute too.

non_english_descriptions['ITEM_DESCRIPTION_ENG'] = non_english_descriptions['ITEM_DESCRIPTION'].apply(to_english)
non_english_descriptions2 = non_english_descriptions.drop(columns=['ITEM_DESCRIPTION']).rename(columns={"ITEM_DESCRIPTION_ENG":"ITEM_DESCRIPTION"})


In [10]:
descriptions2 = pd.concat([non_english_descriptions2,english_descriptions]).sort_index()

In [11]:
descriptions3 = descriptions2[descriptions2['ITEM_DESCRIPTION'] != "null"]

In [12]:
descriptions3

,ASIN,MARKETPLACE_NAME,SELLER_NAME,detected_lang,ITEM_DESCRIPTION
0,B0847TJHB1,United Kingdom,Espring,en,<b>&#10031&#10031 WINTEX Fabric Template Ruler...
2,B01M8GI87J,Germany,Espring,de,<p><b>&#10031&#10031 PRETEX SET OF 5 MICROFIBE...
12,B078X5N2K1,Netherlands,H&S,nl,H&S Foot Massager for Plantar Fasciitis and Ar...
14,B07PQQRNBM,Germany,Espring,de,<b>&#10031&#10031 GOURMEO MUFFIN CUPS &#10031&...
17,B00TY1HA18,United Kingdom,H&S,en,These swivel &amp; braked castors with 4 bolt ...
...,...,...,...,...,...
8630,B01K1135UO,Germany,H&S,de,<b>DOOR STOPPERS THAT ADD A TOUCH OF ELEGANCE ...
8632,B00DEUK2SI,United Kingdom,H&S,en,An exquisite high quality faux crocodile leath...
8633,B0857K4ZDQ,Spain,Espring,es,???? WINTEX transparent wall ???? WINTEX trans...
8637,B07NTY3D93,Germany,Tritart,de,<b>TRITART wipeable board stickers - self-adhe...


In [ ]:
#remove HTML Tags
descriptions3['ITEM_DESCRIPTION2'] = descriptions3['ITEM_DESCRIPTION'].str.replace(r'<[^<>]*>', '', regex=True)

In [ ]:
#keep word characters only
descriptions3['ITEM_DESCRIPTION3'] = descriptions3['ITEM_DESCRIPTION2'].str.replace(r'\W', ' ', regex=True)

In [15]:
descriptions3['MARKETPLACE_NAME'].unique()

array(['United Kingdom', 'Germany', 'Netherlands', 'Italy',
       'United States', 'Spain', 'Sweden', 'Poland', 'Turkey', 'Canada',
       'Belgium'], dtype=object)

In [ ]:
map_marketplace = dict(zip(['United Kingdom', 'Germany', 'Netherlands', 'Italy',
       'United States', 'Spain', 'Sweden', 'Poland', 'Turkey', 'Canada',
       'Belgium'],
    ['UK', 'DE', 'NL','IT','US', 'ES', 'SE','PL','TR','CA','BE']))

descriptions3['MARKETPLACE_NAME'] = descriptions3['MARKETPLACE_NAME'].map(map_marketplace)

In [17]:
descriptions3 = descriptions3.rename(columns={'ASIN1':'ASIN',
                              'MARKETPLACE_NAME':"MARKETPLACE",
                              "SELLER_NAME":"BRAND_NAME",
                                            "ITEM_DESCRIPTION3":"DESCRIPTION"})


In [18]:
final_descriptions = descriptions3[['ASIN','MARKETPLACE','BRAND_NAME','DESCRIPTION']]

In [19]:
final_descriptions

,ASIN,MARKETPLACE,BRAND_NAME,DESCRIPTION
0,B0847TJHB1,UK,Espring,10031 10031 WINTEX Fabric Template Ruler ...
2,B01M8GI87J,DE,Espring,10031 10031 PRETEX SET OF 5 MICROFIBER CLOT...
12,B078X5N2K1,NL,H&S,H S Foot Massager for Plantar Fasciitis and Ar...
14,B07PQQRNBM,DE,Espring,10031 10031 GOURMEO MUFFIN CUPS 10031 10...
17,B00TY1HA18,UK,H&S,These swivel amp braked castors with 4 bolt ...
...,...,...,...,...
8630,B01K1135UO,DE,H&S,DOOR STOPPERS THAT ADD A TOUCH OF ELEGANCE TO ...
8632,B00DEUK2SI,UK,H&S,An exquisite high quality faux crocodile leath...
8633,B0857K4ZDQ,ES,Espring,WINTEX transparent wall WINTEX trans...
8637,B07NTY3D93,DE,Tritart,TRITART wipeable board stickers self adhesiv...


In [20]:
final_descriptions.to_csv('preprocessed_data/Product_Descriptions.csv',index=False)